In [113]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import random
from collections import Counter, defaultdict
from operator import itemgetter
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate
from keras.models import load_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import re
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [31]:
%cd 'mimic-iii-clinical-database-1.4'
%ls

[Errno 2] No such file or directory: 'mimic-iii-clinical-database-1.4'
/Users/jbarman/Documents/CS 598 DLH/DLH_Final_Project/mimic-iii-clinical-database-1.4
ADMISSIONS.csv.gz          LABEVENTS.csv.gz
CALLOUT.csv.gz             LICENSE.txt
CAREGIVERS.csv.gz          MICROBIOLOGYEVENTS.csv.gz
CHARTEVENTS.csv.gz         NOTEEVENTS.csv.gz
CPTEVENTS.csv.gz           OUTPUTEVENTS.csv.gz
DATETIMEEVENTS.csv.gz      PATIENTS.csv.gz
DIAGNOSES_ICD.csv.gz       PRESCRIPTIONS.csv.gz
DRGCODES.csv.gz            PROCEDUREEVENTS_MV.csv.gz
D_CPT.csv.gz               PROCEDURES_ICD.csv.gz
D_ICD_DIAGNOSES.csv.gz     README.md
D_ICD_PROCEDURES.csv.gz    SERVICES.csv.gz
D_ITEMS.csv.gz             SHA256SUMS.txt
D_LABITEMS.csv.gz          TRANSFERS.csv.gz
ICUSTAYS.csv.gz            checksum_md5_unzipped.txt
INPUTEVENTS_CV.csv.gz      checksum_md5_zipped.txt
INPUTEVENTS_MV.csv.gz


In [32]:
import pandas as pd
import os
fn = 'NOTEEVENTS.csv.gz'
df = pd.read_csv(fn, compression='gzip')
df.head()

/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


Data Cleaning

In [59]:
fn = 'DIAGNOSES_ICD.csv.gz'
df_two = pd.read_csv(fn, compression='gzip')
df_two.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [60]:
df_two = df_two[~df_two['ICD9_CODE'].astype(str).str.startswith('V')]

In [61]:
count = df_two.groupby('ICD9_CODE')['SUBJECT_ID'].nunique().nlargest(20)
print(count)

ICD9_CODE
4019     17613
41401    10775
42731    10271
4280      9843
5849      7687
2724      7465
25000     7370
51881     6719
5990      5779
2720      5335
53081     5272
2859      4993
486       4423
2851      4241
2762      4177
2449      3819
496       3592
99592     3560
0389      3433
5070      3396
Name: SUBJECT_ID, dtype: int64


In [109]:
# df_two.join(df, on='HADM_ID', how='inner', lsuffix='_left', rsuffix='_right')
df_two = pd.merge(df, df_two,  how='inner', on=['HADM_ID','ROW_ID', 'SUBJECT_ID'])
df_two.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,...,CHARTDATE_y,CHARTTIME_y,STORETIME_y,CATEGORY_y,DESCRIPTION_y,CGID_y,ISERROR_y,TEXT_y,SEQ_NUM,ICD9_CODE
0,13909,1198,151608.0,2142-06-01,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2142-06-01,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2142-5-23**] Discharge...,8.0,78057
1,15950,1397,118027.0,2176-02-26,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2176-02-26,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2176-2-26**] Discharge...,2.0,1628
2,65519,5823,165240.0,2169-03-15,NaN,NaN,Echo,Report,NaN,NaN,...,2169-03-15,NaN,NaN,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: Pericar...,2.0,340
3,317451,28067,169880.0,2115-04-05,2115-04-05 07:17:00,2115-04-05 07:17:14,Physician,Physician Resident Progress Note,15235.0,NaN,...,2115-04-05,2115-04-05 07:17:00,2115-04-05 07:17:14,Physician,Physician Resident Progress Note,15235.0,NaN,Chief Complaint: Respiratory failure\n 24 Ho...,4.0,4809
4,334126,29299,162468.0,2192-09-25,2192-09-25 09:41:00,2192-09-25 12:51:20,Nursing,Nursing Transfer Note,15623.0,NaN,...,2192-09-25,2192-09-25 09:41:00,2192-09-25 12:51:20,Nursing,Nursing Transfer Note,15623.0,NaN,`\n Pt 55 y/o m who presented from [**Hospit...,12.0,99631


In [110]:
dat = {'HADM_ID', 'SUBJECT_ID', 'DATE', 'ICD9_CODE', 'TEXT'}
data = pd.DataFrame(dat) 
data['HADM_ID'] = df_two['HADM_ID']
data['SUBJECT_ID'] = df_two['SUBJECT_ID']
data['DATE'] = df_two['CHARTDATE']
data['ICD9_CODE'] = df_two['ICD9_CODE']
data['TEXT'] = df_two['TEXT']
data = data.iloc[: , 1:]
data.head()

,HADM_ID,SUBJECT_ID,DATE,ICD9_CODE,TEXT
0,151608.0,1198,2142-06-01,78057,Admission Date: [**2142-5-23**] Discharge...
1,118027.0,1397,2176-02-26,1628,Admission Date: [**2176-2-26**] Discharge...
2,165240.0,5823,2169-03-15,340,PATIENT/TEST INFORMATION:\nIndication: Pericar...
3,169880.0,28067,2115-04-05,4809,Chief Complaint: Respiratory failure\n 24 Ho...
4,162468.0,29299,2192-09-25,99631,`\n Pt 55 y/o m who presented from [**Hospit...


In [63]:
def replace_with_grandparent_codes(string_codes, ICD9_FIRST_LEVEL):
    """replace_with_grandparent_codes takes a list of ICD9 codes and 
    returns the list of their grandparents ICD9 code in the first level of the ICD9 hierarchy"""
    ICD9_RANGES = [x.split('-') for x in ICD9_FIRST_LEVEL]
    resulting_codes = []
    for code in string_codes.split(' '):
        for i,gparent_range in enumerate(ICD9_RANGES):
            range = gparent_range[1] if len(gparent_range) == 2 else gparent_range[0]
            if code[0:3] <= range:
                resulting_codes.append(ICD9_FIRST_LEVEL[i])
                break 
    return ' '.join (set(resulting_codes))

In [64]:
ICD9_FIRST_LEVEL = [
    '001-139','140-239','240-279','290-319', '320-389', '390-459','460-519', '520-579', '580-629', 
    '630-679', '680-709','710-739', '760-779', '780-789', '790-796', '797', '798', '799', '800-999' ]
N_TOP = len(ICD9_FIRST_LEVEL)
# replacing leave ICD9 codes with the grandparents
data['ICD9_CODE'] = data['ICD9_CODE'].apply(lambda x: replace_with_grandparent_codes(x,ICD9_FIRST_LEVEL))
data.head(10)

,HADM_ID,SUBJECT_ID,DATE,ICD9_CODE,TEXT
0,172335,109,2151-08-04,390-459,Admission Date: [**2151-7-16**] Dischar...
1,172335,109,2118-06-14,460-519,Admission Date: [**2118-6-2**] Discharg...
2,172335,109,2119-05-25,580-629,Admission Date: [**2119-5-4**] D...
3,172335,109,2124-08-18,580-629,Admission Date: [**2124-7-21**] ...
4,172335,109,2162-03-25,390-459,Admission Date: [**2162-3-3**] D...


In [65]:
def vectorize_icd_string(x, code_list):
    r = []
    for code in code_list:
        if code in x: r.append(1)
        else: r.append(0)
    return np.asarray(r)

In [66]:
def vectorize_icd_column(df, col_name, code_list):
    r = df[col_name].apply(lambda x: vectorize_icd_string(x, code_list))
    r = np.transpose(np.column_stack(r))
    return r

In [67]:
top_codes = ICD9_FIRST_LEVEL
labels = vectorize_icd_column(data, 'ICD9_CODE', top_codes)
print ('sample of vectorized icd9 labels: ', labels[0])

sample of vectorized icd9 labels:  [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [68]:
from sklearn.model_selection import train_test_split
def train_val_test_split(X, y, val_size=0.2, test_size=0.2, random_state=101):
    """Splits the input and labels into 3 sets"""
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=(val_size+test_size), random_state=random_state)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=test_size/(val_size+test_size), random_state=random_state)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [92]:
# X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(
#     data, labels, val_size=0.2, test_size=0.1, random_state=101)
# print("Train: ", X_train.shape, y_train.shape)
# print("Validation: ", X_val.shape, y_val.shape)
# print("Test: ", X_test.shape, y_test.shape)

In [70]:
# Vectorize ICD codes
def vectorize_icd_string(x, code_list):
    """Takes a string with ICD codes and returns an array of the right of 0/1"""
    r = []
    for code in code_list:
        if code in x: r.append(1)
        else: r.append(0)
    return np.asarray(r)

def vectorize_icd_column(df, col_name, code_list):
    """Takes a column and applies the """
    r = df[col_name].apply(lambda x: vectorize_icd_string(x, code_list))
    r = np.transpose(np.column_stack(r))
    return r


# Clean Text
def clean_str(string):
    """Cleaning of notes"""
    string = string.lower()
    string = string.replace("\n", " ") # remove the lines
    string = re.sub("\[\*\*.*?\*\*\]", "", string) # remove the things inside the [** **]
    string = re.sub("[^a-zA-Z0-9\ \']+", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"(\d+)", "DG", string)
    
    return string.strip()

def clean_notes(df, col_name):
    r = df[col_name].apply(lambda x: clean_str(x))
    return r

def vectorize_notes(col, MAX_NB_WORDS, verbose = True):
    """Takes a note column and encodes it into a series of integer
        Also returns the dictionnary mapping the word to the integer"""
    tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
    tokenizer.fit_on_texts(col)
    data = tokenizer.texts_to_sequences(col)
    note_length =  [len(x) for x in data]
    vocab = tokenizer.word_index
    MAX_VOCAB = len(vocab)
    if verbose:
        print('Vocabulary size: %s' % MAX_VOCAB)
        print('Average note length: %s' % np.mean(note_length))
        print('Max note length: %s' % np.max(note_length))
    return data, vocab, MAX_VOCAB

def pad_notes(data, MAX_SEQ_LENGTH):
    data = pad_sequences(data, maxlen = MAX_SEQ_LENGTH)
    return data, data.shape[1]

# Creates an embedding Matrix
# Based on https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

def embedding_matrix(f_name, dictionary, EMBEDDING_DIM, verbose = True, sigma = None):
    """Takes a pre-trained embedding and adapts it to the dictionary at hand
        Words not found will be all-zeros in the matrix"""

    # Dictionary of words from the pre trained embedding
    pretrained_dict = {}
    with open(f_name, 'r') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            pretrained_dict[word] = coefs

    # Default values for absent words
    if sigma:
        pretrained_matrix = sigma * np.random.rand(len(dictionary) + 1, EMBEDDING_DIM)
    else:
        pretrained_matrix = np.zeros((len(dictionary) + 1, EMBEDDING_DIM))
    
    # Substitution of default values by pretrained values when applicable
    for word, i in dictionary.items():
        vector = pretrained_dict.get(word)
        if vector is not None:
            pretrained_matrix[i] = vector

    if verbose:
        print('Vocabulary in notes:', len(dictionary))
        print('Vocabulary in original embedding:', len(pretrained_dict))
        inter = list( set(dictionary.keys()) & set(pretrained_dict.keys()) )
        print('Vocabulary intersection:', len(inter))

    return pretrained_matrix, pretrained_dict

In [71]:
#preprocess notes
MAX_VOCAB = None # to limit original number of words (None if no limit)
MAX_SEQ_LENGTH = 100 # to limit length of word sequence (None if no limit)
data.TEXT = clean_notes(data, 'TEXT')
data_vectorized, dictionary, MAX_VOCAB = vectorize_notes(data.TEXT, MAX_VOCAB, verbose = True)
data_process, MAX_SEQ_LENGTH = pad_notes(data_vectorized, MAX_SEQ_LENGTH)

print("Final Vocabulary: %s" % MAX_VOCAB)
print("Final Max Sequence Length: %s" % MAX_SEQ_LENGTH)

Vocabulary size: 1941
Average note length: 1645.4
Max note length: 2642
Final Vocabulary: 1941
Final Max Sequence Length: 100


In [72]:
EMBEDDING_DIM = 100 # given the glove that we chose
EMBEDDING_MATRIX= []

#creating glove embeddings
EMBEDDING_LOC = '../notes_100.txt' # location of embedding
EMBEDDING_MATRIX, embedding_dict = embedding_matrix(EMBEDDING_LOC,dictionary, EMBEDDING_DIM, verbose = True, sigma=True)

Vocabulary in notes: 1941
Vocabulary in original embedding: 0
Vocabulary intersection: 0


In [93]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(
    data_process, labels, val_size=0.2, test_size=0.1, random_state=101)
print("Train: ", X_train.shape, y_train.shape)
print("Validation: ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)

Train:  (3, 100) (3, 19)
Validation:  (1, 100) (1, 19)
Test:  (1, 100) (1, 19)


Baseline

In [73]:
data

,HADM_ID,SUBJECT_ID,DATE,ICD9_CODE,TEXT
0,172335,109,2151-08-04,390-459,admission date discharge date service addendum...
1,172335,109,2118-06-14,460-519,admission date discharge date date of birth se...
2,172335,109,2119-05-25,580-629,admission date discharge date service cardioth...
3,172335,109,2124-08-18,580-629,admission date discharge date service medicine...
4,172335,109,2162-03-25,390-459,admission date discharge date date of birth se...


In [74]:
icd9_codes = Counter()
for label in data['ICD9_CODE']:
    for icd9_code in label.split():
        icd9_codes[icd9_code] += 1
number_icd9_first_level = len (icd9_codes)

In [77]:
top_4_icd9 = icd9_codes.most_common(4)
print("most common 4 icd9_codes: ", top_4_icd9)
top_4_icd9_label = ' '.join(code for code,count in top_4_icd9 )
print('label for the top 4 icd9 codes: ', top_4_icd9_label)

most common 4 icd9_codes:  [('390-459', 2), ('580-629', 2), ('460-519', 1)]
label for the top 4 icd9 codes:  390-459 580-629 460-519


In [79]:
#converting ICD9 prediction to a vector
top4_icd9_vector =  vectorize_icd_string(top_4_icd9_label, ICD9_FIRST_LEVEL)
print(ICD9_FIRST_LEVEL)
print(top4_icd9_vector)

['001-139', '140-239', '240-279', '290-319', '320-389', '390-459', '460-519', '520-579', '580-629', '630-679', '680-709', '710-739', '760-779', '780-789', '790-796', '797', '798', '799', '800-999']
[0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0]


In [82]:
def write_col(df, col_name, fname):
    df[col_name].to_csv(fname)
    

def get_f1_score(y_true,y_hat,threshold, average):
    hot_y = np.where(np.array(y_hat) > threshold, 1, 0)
    return f1_score(np.array(y_true), hot_y, average=average)

def show_f1_score(y_train, pred_train, y_val, pred_dev):
    print('F1 scores')
    print('threshold | training | dev  ')
    f1_score_average = 'micro'
    for threshold in [ 0.02, 0.03,0.04,0.05,0.055,0.058,0.06, 0.08, 0.1,0.2,0.3, 0.4, 0.5, 0.6,0.7]:
        train_f1 = get_f1_score(y_train, pred_train,threshold,f1_score_average)
        dev_f1 = get_f1_score(y_val, pred_dev,threshold,f1_score_average)
        print('%1.3f:      %1.3f      %1.3f' % (threshold,train_f1, dev_f1))

In [85]:
## assign icd9_prediction_vector to every discharge
train_y_hat_baseline = [top4_icd9_vector]* len(y_train)
dev_y_hat_baseline = [top4_icd9_vector]* len(y_val)
show_f1_score(y_train, train_y_hat_baseline, y_val, dev_y_hat_baseline)


F1 scores
threshold | training | dev  
0.020:      0.500      0.500
0.030:      0.500      0.500
0.040:      0.500      0.500
0.050:      0.500      0.500
0.055:      0.500      0.500
0.058:      0.500      0.500
0.060:      0.500      0.500
0.080:      0.500      0.500
0.100:      0.500      0.500
0.200:      0.500      0.500
0.300:      0.500      0.500
0.400:      0.500      0.500
0.500:      0.500      0.500
0.600:      0.500      0.500
0.700:      0.500      0.500


In [133]:
print(classification_report(y_train, train_y_hat_baseline))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.33      1.00      0.50         1
           7       0.00      0.00      0.00         0
           8       0.67      1.00      0.80         2
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          17       0.00    

/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [134]:
accuracy_score(y_train, train_y_hat_baseline)

0.0

CNN Model

In [86]:
def build_icd9_cnn_model(input_seq_length, 
                         max_vocab, external_embeddings, embedding_dim, embedding_matrix,
                         num_filters, filter_sizes,
                         training_dropout_keep_prob,
                         num_classes):
    #Embedding
    model_input = Input(shape=(input_seq_length, ))
    if external_embeddings:
        # use embedding_matrix 
        z = Embedding(max_vocab + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=input_seq_length,
                            trainable=True)(model_input)
    else:
        # train embeddings 
        z =  Embedding(max_vocab + 1, 
                   embedding_dim, 
                   input_length=input_seq_length, embeddings_regularizer=regularizers.l2(0.0001),
                   name="embedding")(model_input)

    # Convolutional block
    conv_blocks = []
    for sz in filter_sizes:
        conv = Convolution1D(filters=num_filters,                         
                         kernel_size=sz,
                         padding="valid",
                         activation="relu",
                         strides=1)(z)
        window_pool_size =  input_seq_length  - sz + 1 
        conv = MaxPooling1D(pool_size=window_pool_size)(conv)  
        conv = Flatten()(conv)
        conv_blocks.append(conv)

    #concatenate
    z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
    z = Dropout(training_dropout_keep_prob)(z)

    #score prediction
    model_output = Dense(num_classes, activation="sigmoid")(z)

    #creating model
    model = Model(model_input, model_output)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    print(model.summary())

    return model

In [87]:
model = build_icd9_cnn_model(input_seq_length=MAX_SEQ_LENGTH, max_vocab = MAX_VOCAB,
                             external_embeddings = False,
                             embedding_dim=EMBEDDING_DIM,embedding_matrix=EMBEDDING_MATRIX,
                             num_filters = 100, filter_sizes=[2,3,4,5],
                             training_dropout_keep_prob=0.5,
                             num_classes=N_TOP )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 100)     194200      ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 99, 100)      20100       ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 98, 100)      30100       ['embedding[0][0]']              
                                                                                              

In [94]:
model.fit(X_train, y_train, batch_size=50, epochs=20, validation_data=(X_val, y_val), verbose=2)

Epoch 1/20
1/1 - 2s - loss: 0.7049 - accuracy: 0.0000e+00 - val_loss: 0.6884 - val_accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 2/20
1/1 - 0s - loss: 0.6799 - accuracy: 0.0000e+00 - val_loss: 0.6748 - val_accuracy: 0.0000e+00 - 42ms/epoch - 42ms/step
Epoch 3/20
1/1 - 0s - loss: 0.6610 - accuracy: 0.0000e+00 - val_loss: 0.6617 - val_accuracy: 0.0000e+00 - 37ms/epoch - 37ms/step
Epoch 4/20
1/1 - 0s - loss: 0.6309 - accuracy: 0.3333 - val_loss: 0.6488 - val_accuracy: 0.0000e+00 - 38ms/epoch - 38ms/step
Epoch 5/20
1/1 - 0s - loss: 0.6156 - accuracy: 0.3333 - val_loss: 0.6359 - val_accuracy: 0.0000e+00 - 36ms/epoch - 36ms/step
Epoch 6/20
1/1 - 0s - loss: 0.5752 - accuracy: 0.6667 - val_loss: 0.6231 - val_accuracy: 0.0000e+00 - 28ms/epoch - 28ms/step
Epoch 7/20
1/1 - 0s - loss: 0.5507 - accuracy: 0.6667 - val_loss: 0.6105 - val_accuracy: 0.0000e+00 - 37ms/epoch - 37ms/step
Epoch 8/20
1/1 - 0s - loss: 0.5239 - accuracy: 0.6667 - val_loss: 0.5979 - val_accuracy: 0.0000e+00 - 35ms/epoch - 35

In [95]:
pred_train = model.predict(X_train, batch_size=50)
pred_dev = model.predict(X_val, batch_size=50)
# perform evaluation
show_f1_score(y_train, pred_train, y_val, pred_dev)

F1 scores
threshold | training | dev  
0.020:      0.100      0.100
0.030:      0.100      0.100
0.040:      0.100      0.100
0.050:      0.100      0.100
0.055:      0.100      0.100
0.058:      0.100      0.100
0.060:      0.100      0.100
0.080:      0.100      0.100
0.100:      0.100      0.100
0.200:      0.375      0.100
0.300:      0.857      0.000
0.400:      0.857      0.000
0.500:      1.000      0.000
0.600:      0.800      0.000
0.700:      0.500      0.000


In [128]:
pred_train1 = pred_train.astype('int64')
target_names = ['class 0', 'class 1', 'class 2']

In [132]:
print(classification_report(y_train, pred_train1))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          17       0.00    

/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_s

In [123]:
accuracy_score(y_train, pred_train1)

0.0

In [124]:
y_train

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [126]:
pred_train1

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])